# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - ответить на 4 вопроса:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**
Исследование пройдет в 3 этапа:
1. Обзор данных.
2. Предобработка данных.
3. Ответы на вопросы.

**Описание данных**
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

## Чтение файла с данными и изучение общей информации

In [1]:
#импорт библиотек
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
#чтение файла
data = pd.read_csv('/datasets/data.csv') 
#получение общей информации по таблице
data.info() 
#получение первый 10 строк таблицы
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
#детальнее изучим таблицу и найдем артефакты
data.describe() 

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

* Глядя на таблицу в глаза бросаются `days_employed` и `total_income` у них одинаковое, но разное по отношению к остальным к-во пропусков, обработаем и заполним их.
* Неудобный тип данных у `days_employed` и `total_income`, переведем в int, тем самым и округлим
* Визуально в таблице бросаются в глаза отрицательные значение в столбце `days_employed`, возьмем по модулю методом `.abs`
* Разный регистр в столбце `education`, приведем все к нижнему регистру методом `.str.lower`
* Отрицательное  к-во в столбце  `children`, вероятнее просто ошибка в данных, возьмем по модулю.

## Шаг 2. Предобработка данных

### Обработка пропусков

Найдем к-во пропусков и заодно проверим одни и те же ли эти строки

In [5]:
#найдем к-во пропусков
print(data.isna().sum())
#одинаковые ли это строки:
data[(data['days_employed'].isnull() == True) & (data['total_income'].isnull() == True)].info()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_inc

Это действительно одни и те же строки.
Можно заменить на медианные значения, но зарплаты у каждой профессии разные, поэтому проверим к каким профессиям относятся пропуски и заменим медианными значениями по каждой профессии

In [6]:
#поиск по все профессиям
data[(data['days_employed'].isnull() == True) & (data['total_income'].isnull() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Найдем медианные значения дохода по каждой профессии с сохранением исходного формата датафрейма методом `.transform()` и поместим в новый столбец `median_income`

In [7]:
data['median_income'] = data.groupby('income_type')['total_income'].transform('median')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412


Заменим пропуски медианными значениями по каждой профессии

In [10]:
data['total_income'] = data['total_income'].fillna(data['median_income'])
# Проверим пропуски
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
median_income          0
dtype: int64

С пропусками в `total_income` разобрались, высчитали медиану по каждой профессии и заполнили пропуски.

Осталось разобраться с `days_employed`, возьмем эти значения по модулю методом `.abs()`, т.к отрицательные значения это вероятнее всего ошибка и будет достаточно перевести их в положительные и посчитаем медиану 

In [11]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
median_income          0
dtype: int64

In [12]:
#Найдем медиану и заполним пропуски
days_employed_mean = data['days_employed'].mean()
data['days_employed'] = data['days_employed'].fillna(days_employed_mean)
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
median_income       0
dtype: int64

**Вывод**

Все пропуски заполнены.
* Зарплата была заполнен медианными значениями по каждой профессии
* Стаж нас особо не понадобится, поэтому просто заполнил медианными значениями

### Замена типа данных

In [14]:
#Перевод в int
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')


**Вывод**

* Т.к изначально `days_employed` и `total_income` имеют тип `float` заменили его на `int`, методом `astype('int')`, потому что в нужный тип значения переводят именно этим методом

### Обработка дубликатов

Найдем явные дубликаты методом  `.duplicated().sum()` и обработаем явные дубликаты методом `.drop_duplicates()`

In [15]:
#поиск к-ва явных дубликатов
data.duplicated().sum() 

#удаление
data = data.drop_duplicates().reset_index(drop=True) # и сразу удалим их
print('Число дубликатов:', data.duplicated().sum())
print('Дубликаты удалены')

Число дубликатов: 0
Дубликаты удалены


Найдем дубликаты связанные с регистром столбца `education` методом `.value_counts()` и обработаем их путем замены регистра

In [16]:
#тут только проблемы с регистром, исправим это
data['education'].value_counts() 

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [17]:
# Привели к нижнему регистру
data['education'] = data['education'].str.lower()

# Результат
data['education'].value_counts() 

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Поищем остальные дубликаты методом `value_counts()` который находит уникальные значения и возвращает частоту его встречаемости

In [18]:
data['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

С `family_status` все в порядке.

In [19]:
data['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

С `purpose` все не так радужно, поможет лемматизация

**Вывод**

* При обработке нашли и удалили 71 явный дубликат, скорее всего это просто совпадение, поэтому удалаяем методом `drop_duplicates()` т.к он просто удаляет одинаковые строки, что нам подходит
* Были найдены дубликаты связанные с разным регистром столбца `education`, это просто человеческий фактор и чтобы его избежать стоит добавить выпадающий список или правило как правильно заполнять. Дубликаты были найдены методом `value_counts()` ,который выбирает уникальное значение и подсчитывает частоту возникновения, а исправлены путем приведения всего к нижнему регистру методом `str.lower()`

### Лемматизация

In [25]:
#Просчитаем все цели кредита.
data['purpose'].value_counts() 

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Из всего списка можно выбрать не так много ключевых слов, а именно: свадьба, автомобиль, образование, коммерческий, жилье, недвижимость, сдача

In [26]:
#Ключевые слова
categories = ['сдача', 'свадьба', 'автомобиль', 'образование', 'коммерческий', 'жилье', 'недвижимость']

#Лемматизация
def lemmatize (text):
    lemma = m.lemmatize(text)
    for word in categories:
        if word in lemma:
            lemma = word
    return lemma
data['purpose_category'] = data['purpose'].apply(lemmatize)

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income,income_category,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,142594.396847,very_big_income,жилье
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,142594.396847,normal_income,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,142594.396847,big_income,жилье
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,142594.396847,very_big_income,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,118514.486412,big_income,свадьба


In [27]:
#Подсчет к-ва категорий
data['purpose_category'].value_counts()

недвижимость    5041
автомобиль      4308
образование     4014
жилье           3809
свадьба         2335
коммерческий    1312
сдача            652
Name: purpose_category, dtype: int64

Немного подкорректируем список:
* Жилье и недвижимость объединим в одну категорию
* Коммерческий и сдачу объединим в категорию `коммерция`

In [28]:
data.loc[data['purpose_category'] == 'жилье', 'purpose_category'] = 'недвижимость'
data.loc[(data['purpose_category'] == 'коммерческий') | (data['purpose_category'] == 'сдача'), 'purpose_category'] = 'коммерция'

In [29]:
data['purpose_category'].value_counts()

недвижимость    8850
автомобиль      4308
образование     4014
свадьба         2335
коммерция       1964
Name: purpose_category, dtype: int64

**Вывод**

Выделили ключевые слова и для удобства обработали их:
* "Недвижимость" и "жилье" вынесли в одну группу `недвижимость` т.к эти обе группы подразумевают пользователя, целью которого является проживание
* "Коммерческий" и "сдача" вынесли в одну группу `коммерция` т.к эти обе группы подразумевают пользователя, целью которого является извлечение прибыли

### Категоризация данных

Чтобы ответить на вопросы по проекту нам нужно 4 группы:
* Категоризация по целям
* Категоризация по доходам
* Категоризация по детям
* Категоризация по семейному положению

#### Категоризация по доходам
Разобьем на интервалы равного значения методом `.qcut()`

In [30]:
#Возьмем проценты и разобьем на 4 категории в соотношении
stats = data['total_income'].describe()
stats[4:-1] 

25%    107654.5
50%    142594.0
75%    195767.5
Name: total_income, dtype: float64

In [31]:
def income_categories(data):
    if data <= stats[4]:
        return 'small_income'
    elif stats[4] < data <= stats[5]:
        return 'normal_income'
    elif stats[5] < data <= stats[6]:
        return 'big_income'
    else:
        return 'very_big_income'

data['income_category'] = data['total_income'].apply(income_categories)
data.head()
#data['income_categories'].value_counts()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income,income_category,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,142594.396847,very_big_income,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,142594.396847,normal_income,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,142594.396847,big_income,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,142594.396847,very_big_income,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,118514.486412,big_income,свадьба


Разбили уровень доходов на 4 категории :
* `small_income` - доходы ниже 107 654
* `normal_income` - доходы в диапазоне 107 654 : 150 427
* `big_income` - доходы в диапазоне 150 427 : 195 767
* `very_big_income` - доходы свыше 195 767

Этих данных нам хватит что ответить на вопрос `Есть ли зависимость между уровнем дохода и возвратом кредита в срок?`

#### Категоризация по целям.
Этот словарь у нас был сделан при лемматизации. Категориями будут выступать сами цели.

In [32]:
data['purpose_category'].value_counts()

недвижимость    8850
автомобиль      4308
образование     4014
свадьба         2335
коммерция       1964
Name: purpose_category, dtype: int64

Этих данных нам хватит, чтобы ответить на вопрос `Как разные цели кредита влияют на его возврат в срок?`

#### Категоризация по детям.

In [33]:
#data['children'].value_counts() # исправим недарозумение с -1 и 20 детей, вероятнее всего это ошбика.
# -1 уберем взятием по модулю, это не повредит остальным данным
data['children'] = data['children'].abs() 
# Заменим 20 дейтей двумя, т.к тоже скорее всего ошибка
data.loc[data['children'] == 20, 'children'] = 2 
# Проверим еще раз
data['children'].value_counts() 

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Нет смысла категоризировать этот пункт, он и без нашего участия категоризирован по к-ву детей

#### Категоризация по семейному положению

In [34]:
data['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

По аналогии с пунктом `2.5.3`

**Вывод**

Провели категоризацию и вывели 2 столбца `income_category` и `purpose_category`. Ответы на остальные вопросы уже имеются в таблице и не требуют категоризации:
* Семейных положений не так много и каждый является своей категорией, всего 5 категорий
* К-во детей так же является категориями и их всего 6

Всех этих данным нам вполне хватит, чтобы ответить на требуемые вопросы.

<div class="alert alert-success">

<b>🔁1️⃣ Ревьюер: 👍</b> Да, все верно и вывод хороший.

</div>

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [35]:
#Сводная таблица, где отражется сколько должников и платящих приходится на каждую группу детей
children_debt_pivot = data.pivot_table(index=['children'], columns='debt', values='gender', aggfunc='count') 
display(children_debt_pivot) 

debt,0,1
children,,
0,13044.0,1063.0
1,4411.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


In [36]:
# Добавим колонку с соотношением платящих к должникам
children_debt_pivot['ratio'] = (children_debt_pivot[1] / (children_debt_pivot[0] + children_debt_pivot[1]))* 100
# Отсортируем по ratio
children_debt_pivot.sort_values(by='ratio', ascending=False) 
                                                              

debt,0,1,ratio
children,,,
4,37.0,4.0,9.756098
2,1926.0,202.0,9.492481
1,4411.0,445.0,9.163921
3,303.0,27.0,8.181818
0,13044.0,1063.0,7.535266
5,9.0,NaN,NaN


<div class="alert alert-success">

<b>🔁1️⃣ Ревьюер: 👍</b> `ratio` можно было умножить на сто, чтобы получить проценты. 

</div>

**Вывод**

***Процент невозврата***
* Без детей - 7.5%
* С одним ребенком - 9.2%
* С двумя - 9.5%
* С тремя - 8.2%
* С четырьмя - 9.8%

Отсюда можно сделать вывод:
* Однозначно наличие детей влияет на возврат кредита в срок, ведь вряд ли кто-то оставит ребенка голодным, чтобы заплатить за кредит
* Но судя по этим данным нельзя сказать, что процент невозврата растет пропорционально к к-ву детей, но вероятнее всего это из-за слишком малой выборки.

<div class="alert alert-success">

<b>🔁1️⃣ Ревьюер: 👍</b> Абсолютно верно.

</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [37]:
# Создадим сводную таблицу
family_debt_pivot = data.pivot_table(index=['family_status'], columns='debt', values='gender', aggfunc='count')
display(family_debt_pivot)

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3775,388
женат / замужем,11413,931


In [38]:
# Добавим соотношение и отсортируем по нему
family_debt_pivot['ratio'] = (family_debt_pivot[1] / (family_debt_pivot[0]+family_debt_pivot[1]))*100
family_debt_pivot.sort_values(by='ratio', ascending=False)

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2536,274,9.750890
гражданский брак,3775,388,9.320202
женат / замужем,11413,931,7.542126
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343


**Вывод**

**Процент невозврата**
* Не женат / не замужем - 9.8%
* гражданский брак - 9.3%
* женат / замужем - 7.5%
* в разводе - 7.1%
* вдовец / вдова - 6.6%

Отсюда можно сделать вывод, что люди живущие вне оффициального брака платят реже чем люди в браке или бывшие в нем.
Это обуславливается тем, что:
* Холостяки и состоящие в отношениях чаще тратят деньги на "погулять" и делают больше импульсивных покупок
* А жизнь в браке их остепеняет и учит думать преждем чем тратить

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
#Создадим сводную таблицу
income_category_debt = data.pivot_table(index=['income_category'], columns='debt', values='gender', aggfunc='count')
display(income_category_debt)

debt,0,1
income_category,,
big_income,4797,448
normal_income,5007,483
small_income,4941,427
very_big_income,4985,383


In [40]:
# Добавим соотношение и осортируем по нему:
income_category_debt['ratio'] = (income_category_debt[1] / (income_category_debt[0] + income_category_debt[1]))*100
income_category_debt.sort_values('ratio', ascending=False)

debt,0,1,ratio
income_category,,,
normal_income,5007,483,8.797814
big_income,4797,448,8.541468
small_income,4941,427,7.954545
very_big_income,4985,383,7.134873


**Вывод**

**Процент невозврата**
* normal_income - 8.8%
* big_income - 8.5%
* small_income - 7.9%
* very_big_income - 7.1%

Отсюда делаем вывод:
* Люди с самым большим и самым маленьким доходом чаще всего платят кредит в срок, это обуславливается их финансовой грамотностью и умением распоряжаться своим бюджетом, причем богатые люди меньше всего просрачивают
* Люди со средним и выше среднего доходом, являются лидерами: ведь когда у тебя появляется денег чуть больше чем очень мало(`normal_income`), ты на радостях начинаешь тратить больше, поэтому нехватает денег платить по кредиту, но с ростом доходов (`big_income`) твой пыл охлаждается и ты начинаешь понемногу считать деньги.
    
    * Но этот пункт не относится к категории `small_income`, ведь когда у тебя и так мало денег, у тебя просто нечего тратить и ты считаешь свои "кровные"


- Как разные цели кредита влияют на его возврат в срок?

In [31]:
# Сводная таблица
purpose_category_debt = data.pivot_table(index=['purpose_category'], columns='debt', values='gender', aggfunc='count')
display(purpose_category_debt)

debt,0,1
purpose_category,,
автомобиль,3905,403
коммерция,1813,151
недвижимость,8219,631
образование,3644,370
свадьба,2149,186


In [32]:
# Соотношение и сортировка
purpose_category_debt['ratio'] = (purpose_category_debt[1] / (purpose_category_debt[1] + purpose_category_debt[0]))*100
purpose_category_debt.sort_values('ratio', ascending=False)

debt,0,1,ratio
purpose_category,,,
автомобиль,3905,403,9.354689
образование,3644,370,9.217738
свадьба,2149,186,7.965739
коммерция,1813,151,7.688391
недвижимость,8219,631,7.129944


**Вывод**

**Процент невозврата**
* автомобиль - 9.4%
* образование - 9.2%
* свадьба - 7.9%
* коммерция - 7.7%
* недвижимость - 7.1%

**Отсюда вывод:**
* Меньше всего должников в категории `недвижимость`, это обсулавливается тем, что недвижмость это , как правило, очень дорогостоящяя покупка при малых доходах, отсюда люди более ответственны. Также покупка недвижимости это полностью обдуманное и взвешенное решение, плюс зачастую помогают родственники, отсюда и дисциплина в платежах.
* Больше всего должников в категориях `автомобиль` и `образование`, люди не так серьезно к этому относятся, ведь суммы не прям заоблачные. Еще, возможно, играет фактор статусности, люди хотят машину и образование по престижнее, поэтому немного перебирают с кредитом и не справляются с платежами, по причине не обдуманности решений.



## Шаг 4. Общий вывод

Все 4 категории :
1. Дети
2. Цель кредита
3. Семейное положение
4. Уровень дохода

так или иначе влияют на возврат кредита.

Можно подвести итог:
* Когда у людей есть дети (1, 2, 3, 4 ребенка), они с вероятностью (9.2%, 9.5%, 8.2%, 9.8% соответственно) откладывают платеж по кредиту, против 7.5% у бездетных, пусть первые более ответственные, но детей нужно чем-то кормить.


* Реже всего просрачивают платежи люди, которые берут кредит на `недвижимость` - 7.1%,`коммерцию` - 7.7%,`свадьбу` - 7.9%, потому что их решение взвешенное и обдуманное, они сразу готовы на это, а вот люди которые берут кредит на `автомобиль` - 9.4% и `образование` - 9.2%, чаще всего просрачивают платежи, потому что и машину хочется по новее и ВУЗ по престижние, а денег не всегда на такое хватает.


* `Женатые`, `в разводе` или `вдовцы`, также реже просрачивают кредит(7.5%, 7.1% и 6.6%, соответственно), по причине привычке платить по долгам и с умом относиться к семейному бюджету, в отличие от `холостяков` и `гражданского брака`(9.8% и 9.3%, соответственно).


* `Очень богатые люди` и `люди с маленьким доходам` относятся к деньгам осторожнее и лучше распределяют свой бюджет, соответственно и платят исправнее (7.1% и 7.9%,соответственно) чем те у кого `средний` и `выше среднего` (8.8% и 8.5%, соответственно).

Отсюда можно вывести лучшего и не очень клиента банка:
* Человек без детей, с доходом от 195 тысяч, женатый или бывший в браке, целью которого является коммерческая/жилая недвижимость
* Человек с 1,2 детьми, с доходом  от 107 654 : 195 767, холостой, целью которого является автомобиль

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.